In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ReduceLROnPlateau

In [2]:
X_train = np.load("features_train.npy")
X_test = np.load("features_test.npy")
y_train = np.load("labels_train.npy")
y_test = np.load("labels_test.npy")

In [3]:
DATADIR_TRAIN = "C:/Users/Gurpreet/Documents/BME571 - ML in BME/COVID_CNN/Covid19-dataset/train"
DATADIR_TEST = "C:/Users/Gurpreet/Documents/BME571 - ML in BME/COVID_CNN/Covid19-dataset/test"

In [4]:
X_train = X_train / 255.0
X_test = X_test / 255.0
IMG_SIZE = 200

In [5]:
X_train[1:1]

array([], shape=(0, 200, 200, 3), dtype=float64)

In [6]:
train_generator = ImageDataGenerator(
    rescale=1/255.,              # normalize pixel values between 0-1
    width_shift_range=0.5,       # shift the width of the image 50%
    rotation_range=90,           # random rotation by 90 degrees
    horizontal_flip=True,        # 180 degree flip horizontally
    vertical_flip=True,          # 180 degree flip vertically
    validation_split=0.30        # 15% of the data will be used for validation at end of each epoch
)
test_generator = ImageDataGenerator(
    rescale=1/255.,              # normalize pixel values between 0-1
)

In [7]:
train_generator.fit(X_train)
test_generator.fit(X_test)

In [8]:
train_set = train_generator.flow_from_directory(
                    DATADIR_TRAIN,
                    subset = 'training',
                    batch_size=32,
                    target_size=(IMG_SIZE,IMG_SIZE),
                    class_mode = 'sparse'
)
val_set = train_generator.flow_from_directory(
                    DATADIR_TRAIN,
                    subset = 'validation',
                    batch_size=32,
                    target_size=(IMG_SIZE,IMG_SIZE),
                    class_mode = 'sparse'
)
test_set = test_generator.flow_from_directory(
                    DATADIR_TEST,
                    batch_size=32,
                    target_size=(IMG_SIZE,IMG_SIZE),
                    class_mode = 'sparse'
)

Found 176 images belonging to 3 classes.
Found 75 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [9]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
model = Sequential()

model.add(Conv2D(128, (3,3), input_shape=(IMG_SIZE,IMG_SIZE,3)))
model.add(Activation('relu'))
model.add(Dropout (0.25))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, (3,3)))
model.add(Dropout (0.15))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#model.add(Conv2D(256, (3,3)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(144, activation= 'relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(train_generator.flow(X_train, y_train, batch_size=128),
                        epochs = 3,
                        validation_data = (X_test, y_test),
                        verbose = 2,
                        steps_per_epoch = 7
                        )